### **Northwind** (Gold Modeling)

In [1]:
import json
from notebookutils import mssparkutils
from pyspark.sql import SparkSession, DataFrame, functions as F
from pyspark.sql.window import Window
from delta.tables import DeltaTable
#For writing to Warehouse using Spark Connector 
import com.microsoft.spark.fabric
from com.microsoft.spark.fabric.Constants import Constants

StatementMeta(, feeb413e-b4b9-425d-abef-3ef6d3fae22c, 3, Finished, Available, Finished)

#### **Get Silver Metadata**
Receives Silver Table Metadata from the Silver Notebook via _Data Factory Pipeline_

In [1]:
# Parameterized Value
#silver_metadata = ""

StatementMeta(, 32789a20-2902-423a-9cec-9b86d800aef2, 3, Finished, Available, Finished)

In [ ]:
# Read Pipeline Metadata
records = json.loads(silver_metadata)
results = []

#### **Gold Ingestion**

In [ ]:
for rec in records:

    # Skip tables that failed in Silver
    if rec.get("SilverStatus") != "Success":
        rec["GoldStatus"] = "Skipped"
        continue

    try:
        # Mapping Metadata
        table_name = str(rec["TableName"]).lower().replace(" ", "")
        model_type = rec.get("Modeling", "").lower()
        key_cols = rec["KeyColumn"].split("|") #For multiple key columns scenarios 
        gold_schema = str(rec["DataDomain"]).lower()  
        warehouse_name = "DW_GOLD_northwind" 
        silver_lakehouse = "NorthWind_LK" 

        # Build final dimensional table name (fact/dim/bridge)
        if model_type == "facts":
            dim_table = f"facts_{table_name}"
        elif model_type == "dimensions":
            dim_table = f"dim_{table_name}"
        else:
            dim_table = f"bridge_{table_name}"

        
        # Load Silver Delta Table
        df = spark.read.table(f"{gold_schema}.{table_name}")

        # Write to Warehouse using Spark Connector 
        df.write.mode("overwrite").synapsesql(f"{warehouse_name}.{gold_schema}.{dim_table}")

        # --- Logging
        rec["GoldStatus"] = "Success"
        rec["GoldTableName"] = dim_table

    except Exception as e:
        rec["GoldStatus"] = f"Failed: {str(e)}"


#### **Notebook Return Output**

In [ ]:
# Optional: pretty-print the output
print("Returning payload →", json.dumps(records, indent=2))

# Return full JSON for pipeline/logging use
mssparkutils.notebook.exit(json.dumps(records, indent=4))